In [46]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
import datetime
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations
from scipy.stats import qmc
from bo_functions_generic import gen_theta_set, find_train_doc_path, test_train_split, clean_1D_arrays
import itertools
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations
import random

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [60]:
Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])

Theta_True = Constants[1:3].flatten()
print(Constants.shape)
print(Theta_True.shape)

# print(Theta_True)

d = len(Theta_True)
skip_params=1

obj = np.array(["obj"])
# obj = np.array(["obj","LN_obj"])
# emulator = False
emulator = np.array([True])

#Pull Experimental data from CSV
exp_d = 2
n = 15 #Number of experimental data points to use
exp_data_doc = 'Input_CSVs/Exp_Data/d='+str(exp_d)+'/n='+str(n)+'.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1:exp_d+1]
Yexp = exp_data[:,-1]
print(Xexp.shape)

if emulator == True:
    t = 20*n
else:
    t = 20
    
#Define GP Testing space
LHS = True
p=20
bounds = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
                   [ 2,  2,   0,  2,  2,  2, 15,  2]])
# Theta1 =  np.linspace(-2,2,p) #1x10
# Theta2 =  np.linspace(-2,2,p) #1x10
# theta_mesh = np.array(np.meshgrid(Theta1, Theta2)) #2 Uniform 5x5 arrays 
theta_mesh = gen_theta_set(LHS = LHS, n_points = p, dimensions = d, bounds = bounds)
print(theta_mesh.shape)

all_data_doc = find_train_doc_path(emulator, obj, d, t)
all_data = np.array(pd.read_csv(all_data_doc, header=0,sep=",")) 
print(all_data.shape)

train_data, test_data = test_train_split(all_data, sep_fact = 0.2, shuffle_seed=9)
train_p, test_p = train_data[:,1:-1], test_data[:,1:-1]
# print(train_p)
print(train_p.shape)
print(test_p.shape)




(6, 4)
(8,)
(15, 2)
(400, 8)
(300, 12)
torch.Size([60, 10])
torch.Size([240, 10])


In [61]:
Constants2 = np.split(Constants, [1,3])
print(Constants)
print(Constants2)

[[-200.  -100.  -170.    15. ]
 [  -1.    -1.    -6.5    0.7]
 [   0.     0.    11.     0.6]
 [ -10.   -10.    -6.5    0.7]
 [   1.     0.    -0.5   -1. ]
 [   0.     0.5    1.5    1. ]]
[array([[-200., -100., -170.,   15.]]), array([[-1. , -1. , -6.5,  0.7],
       [ 0. ,  0. , 11. ,  0.6]]), array([[-10. , -10. ,  -6.5,   0.7],
       [  1. ,   0. ,  -0.5,  -1. ],
       [  0. ,   0.5,   1.5,   1. ]])]


In [120]:
Constants_1, Constants_theta, Constants_4_6 = np.split(Constants, [skip_params,3])
print(Constants_1.shape, Constants_theta.shape, Constants_4_6.shape)
# print(Constants_1, Constants_theta, Constants_4_6)

Constants_1_scl, scaler_C1 = norm_unnorm(Constants_1.T, norm = True)

Constants_theta_scl, scaler_theta =  norm_unnorm(clean_1D_arrays(Constants_theta.flatten(), param_clean = True),
                                                 norm = True, scaler = scaler_theta)
Constants_theta_scl = Constants_theta_scl.reshape((2,4))

Constants_4_6_scl, scaler_4_6 = norm_unnorm(Constants_4_6.T, norm = True)
print(Constants_1_scl.shape, Constants_theta_scl.shape, Constants_4_6_scl.shape)

Constants_scl = np.vstack((Constants_1_scl.T, Constants_theta_scl, Constants_4_6_scl.T))
print(Constants_scl)



(1, 4) (2, 4) (3, 4)
(4, 1) (2, 4) (4, 3)
[[0.         0.46511628 0.13953488 1.        ]
 [0.22647837 0.24460017 0.35483686 0.66965647]
 [0.49709093 0.5089776  0.62200386 0.65634922]
 [0.         0.         0.3271028  1.        ]
 [1.         0.5        0.25       0.        ]
 [0.         0.33333333 1.         0.66666667]]


In [113]:
# print(norm_unnorm(Constants_1_scl.T, norm = False, scaler = scaler_C1))
# print(Constants_1)
print(normalize_p_true(Theta_True, scaler_theta))

[0.22647837 0.24460017 0.35483686 0.66965647 0.49709093 0.5089776
 0.62200386 0.65634922]


In [118]:
def normalize_x(train_p, m, Xexp, emulator):
    if emulator == True:
        train_x_scl, scaler_x = norm_unnorm(train_p[:,-m:], norm = True)
    else:
        train_x_scl, scaler_x = norm_unnorm(Xexp[:,-m:], norm = True)
    return train_x_scl, scaler_x

def normalize_p_data(param_vals_data, m, emulator):
    if emulator == True:
        param_data_scl, scaler_theta = norm_unnorm(param_vals_data[:,0:-m], norm = True)
    else:
        param_data_scl, scaler_theta = norm_unnorm(param_vals_data, norm = True)
    return param_data_scl, scaler_theta

def normalize_p_set(p_set, scaler_theta):
    p_set_scl, scaler_theta = norm_unnorm(p_set, norm = True, scaler = scaler_theta)
    return p_set_scl

def normalize_p_true(p_true, scaler_theta):
    theta_true_scl, scaler_theta = norm_unnorm(clean_1D_arrays(p_true, param_clean = True), 
                                               norm = True, scaler = scaler_theta)
    return theta_true_scl.flatten()

def normalize_constants(Constants, p_true, scaler_theta, skip_params, CS):
    num_param_types = clean_1D_arrays(Constants).shape[1]
    len_param_type = int(len(p_true)/num_param_types)
    if CS == 1:
        Constants_scl = normalize_p_true(p_true, scaler_theta)
    else:
        Constants_before, Constants_theta, Constants_after = np.split(Constants, [skip_params,len_param_type+1])
        Constants_before_scl, scaler_C_before = norm_unnorm(Constants_before.T, norm = True)
        
        Constants_theta_scl, scaler_theta =  norm_unnorm(clean_1D_arrays(Constants_theta.flatten(), param_clean = True),
                                                 norm = True, scaler = scaler_theta)
        Constants_theta_scl = Constants_theta_scl.reshape((2,4))
        
        Constants_after_scl, scaler_C_after = norm_unnorm(Constants_after.T, norm = True)
        
        Constants_scl = np.vstack((Constants_before_scl.T, Constants_theta_scl, Constants_after_scl.T))
    return Constants_scl      

[[0.         0.46511628 0.13953488 1.        ]
 [0.22647837 0.24460017 0.35483686 0.66965647]
 [0.49709093 0.5089776  0.62200386 0.65634922]
 [0.         0.         0.3271028  1.        ]
 [1.         0.5        0.25       0.        ]
 [0.         0.33333333 1.         0.66666667]]


In [24]:
from sklearn.preprocessing import MinMaxScaler
random.seed(10)
X = np.random.randint(-10,10, size = (3,3))
Y = np.random.randint(-10,10, size = (3,3))
print(X)
# print(Y)
# from sklearn.model_selection import LeaveOneOut

# y_true  = []
# y_pred = []

# print(loo.get_n_splits(X))
# for train_index, test_index in loo.split(X):
#     df_test = X[test_index]
#     df_train = X[train_index]
#     print(df_train, df_test)
    

# scaler =  MinMaxScaler()
# scaler.fit(X)
# X = scaler.transform(X)
# print(X)
# X = scaler.inverse_transform(X)
# print(X)

from bo_functions_generic import norm_unnorm
norm = True
X_scl, scaler_X = norm_unnorm(X, norm = norm)
Y_scl, scaler_Y = norm_unnorm(Y, norm = norm)

print(X_scl, scaler_X)
# print(Y, scaler_Y)

X2 = np.array([[0, 1, 0.33],
               [0.25 , 0.75, 0.66]])

X3 = np.array([[2, 5, -2],
               [-5 , 4, 0]])
print(X3)
X3, scaler_X = norm_unnorm(X3, norm = True, scaler = scaler_X)
print(X3)

[[-2 -4 -3]
 [ 2  1 -1]
 [ 7  0 -7]]
[[0.         0.         0.66666667]
 [0.44444444 1.         1.        ]
 [1.         0.8        0.        ]] MinMaxScaler()
[[ 2  5 -2]
 [-5  4  0]]
[[ 0.44444444  1.8         0.83333333]
 [-0.33333333  1.6         1.16666667]]


In [50]:
Xexp_bounds = np.array([[-1.5, -0.5],
                        [1 , 2]])
LHS = gen_theta_set(LHS = False, n_points = 20, dimensions = 2, bounds = Xexp_bounds)
# print(LHS)
# print(LHS.reshape((20,20,-1)).T)


X1 = np.linspace(-1.5,1,20)
X2 = np.linspace(-0.5,2,20)
X_mesh = np.meshgrid(X1,X2)
print(np.array(X_mesh))

[[[-1.5        -1.36842105 -1.23684211 -1.10526316 -0.97368421
   -0.84210526 -0.71052632 -0.57894737 -0.44736842 -0.31578947
   -0.18421053 -0.05263158  0.07894737  0.21052632  0.34210526
    0.47368421  0.60526316  0.73684211  0.86842105  1.        ]
  [-1.5        -1.36842105 -1.23684211 -1.10526316 -0.97368421
   -0.84210526 -0.71052632 -0.57894737 -0.44736842 -0.31578947
   -0.18421053 -0.05263158  0.07894737  0.21052632  0.34210526
    0.47368421  0.60526316  0.73684211  0.86842105  1.        ]
  [-1.5        -1.36842105 -1.23684211 -1.10526316 -0.97368421
   -0.84210526 -0.71052632 -0.57894737 -0.44736842 -0.31578947
   -0.18421053 -0.05263158  0.07894737  0.21052632  0.34210526
    0.47368421  0.60526316  0.73684211  0.86842105  1.        ]
  [-1.5        -1.36842105 -1.23684211 -1.10526316 -0.97368421
   -0.84210526 -0.71052632 -0.57894737 -0.44736842 -0.31578947
   -0.18421053 -0.05263158  0.07894737  0.21052632  0.34210526
    0.47368421  0.60526316  0.73684211  0.86842105  

In [51]:
# https://realpython.com/python-timer/
import time
start = 3678393
res = datetime.timedelta(seconds =start)
print(res)

time_tot = 0
x = 0
for i in range(89900):
    start2 = time.time()
    x += 1+2
    end = time.time()

    time_tot+= (end - start2)


print(time_tot)

42 days, 13:46:33
0.016117095947265625


In [26]:
A = np.array([[4,3,0,0],
              [4,0,0,3]])
print(A[np.nonzero(A)])

[4 3 4 3]


In [16]:
B = np.zeros((3,3))
B = pd.DataFrame(B)
type(B)
isinstance(B, pd.DataFrame)

True

In [11]:
np.random.seed(9)
Array_1 = np.random.rand(3,2)
Array_2 = np.random.rand(3,2)

print(Array_1)
print(Array_2)

ds = gdal.Open('image.tif')
# loop through each band

path = 

with open("converted.raw", "wb") as outfile:
    for bi in range(ds.RasterCount):
        band = ds.GetRasterBand(bi + 1)
        # Read this band into a 2D NumPy array
        ar = band.ReadAsArray()
        print('Band %d has type %s'%(bi + 1, ar.dtype))   
        np.save(outfile, ar.astype('uint16'))

[[0.01037415 0.50187459]
 [0.49577329 0.13382953]
 [0.14211109 0.21855868]]
[[0.41850818 0.24810117]
 [0.08405965 0.34549864]
 [0.16677635 0.87855909]]
